<a href="https://colab.research.google.com/github/jouvetg/igm/blob/main/notebooks/IGM_aletsch_1880_2100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### <h1 align="center" id="title">IGM notebook tutorial (aletsch-1880-2100) </h1>

This set-up aims to closely reproduce the simulations of the Great Aletsch Glacier (Switzerland) in the past and in the future based
on the CH2018 climate scenarios described in (Jouvet and al., JOG, 2011) and (Jouvet and Huss, JOG, 2019), respectively. 
The goal of this set-up is to give a template of climate-based mass balance implementation within the TensorFlow framework,
which can serve for future improvements of the Aletsch case, or adaptations to other mountain glaciers. This simulation relies on
an accumulation and distributed temperature-index melt mass balance model (Hock, JOG, 1999). The closest and most recent
description of the implemented model is given in (Jouvet and al., TC, 2020). This simulation does not stricly match the original
implementation due to discrepencies in input data and ice flow models (emulated Stokes vs original Stokes). Therefore, melt
factors were re-calibrated to get the best match between observed and modelled top ice surface in the past. 


Let us firt download the IGM code, ice flow emulator, and input data. Input files include geological inputs (geology.nc), spatially-varying fields for the computation of melt factors and snow reditribution (massbalance.nc), as well as temperature and precipitation time serie data (temp_prec.dat). The data also provide some observed top surface ice topographies (in 1880, 1926, 1957, 1980, 1999, 2009, 2017 and 2016).

In [1]:
# get the core source of igm (in igm.py)
!wget -nv -O  igm.py https://raw.githubusercontent.com/jouvetg/igm/main/src/igm.py

# get input data (topography, mass balance and climate data, resp.)
!wget -nv -O geology.nc https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-1880-2100/geology.nc
!wget -nv -O massbalance.nc https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-1880-2100/massbalance.nc
!wget -nv -O mbparameter.dat https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-1880-2100/mbparameter.dat
!wget -nv -O temp_prec.dat https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-1880-2100/temp_prec.dat

# get specific igm functions for this set-up (climate and mass balance, resp.)
!wget -nv -O igm_clim_aletsch.py https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-1880-2100/igm_clim_aletsch.py
!wget -nv -O igm_smb_accmelt.py https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-1880-2100/igm_smb_accmelt.py

# get the ice flow DL-emulator
!apt install subversion
!svn export https://github.com/jouvetg/igm/trunk/model-lib/f15_cfsflow_GJ_22_a


2022-06-17 07:20:48 URL:https://raw.githubusercontent.com/jouvetg/igm/main/src/igm.py [128518/128518] -> "igm.py" [1]
2022-06-17 07:20:48 URL:https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-1880-2100/geology.nc [1942564/1942564] -> "geology.nc" [1]
2022-06-17 07:20:49 URL:https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-1880-2100/massbalance.nc [1567369/1567369] -> "massbalance.nc" [1]
2022-06-17 07:20:49 URL:https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-1880-2100/mbparameter.dat [511/511] -> "mbparameter.dat" [1]
2022-06-17 07:20:49 URL:https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-1880-2100/temp_prec.dat [2856735/2856735] -> "temp_prec.dat" [1]
2022-06-17 07:20:50 URL:https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-1880-2100/igm_clim_aletsch.py [4044/4044] -> "igm_clim_aletsch.py" [1]
2022-06-17 07:20:50 URL:https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-1880-21

First, we import necessary libraries, check the version of tensorflow, and inport the class igm (defined in igm.py), and create an igm object.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time
import sys

sys.argv = ['']  # this is absolutly necessary in Jupyter notebook

from igm import *
from igm_clim_aletsch import *
from igm_smb_accmelt import *
 
# add extensions for climate generation and mass balance to the core igm class
class igm(igm,igm_clim_aletsch, igm_smb_accmelt):
    pass

# define the igm class
igm = igm()

Altough, you don't have to, it is highly recommended to run under GPU (click above 'Runtime' -> 'Change Runtime type' -> Activate GPU). Then the fowllowing comand check that a GPU is indeed available, and give the name of the GPU. (With the free veryion of colab these GPU are often old and not that efficient -- like the K80 --, IGM would typically run faster on new material)

Next, we overide the default configuration parameters, initialize the class, and check at the parameters

In [3]:
# Set-up parameters

igm.config.working_dir           = ''  
igm.config.tstart                = 1880  # starting time (you may use other available year like 1926)
igm.config.tend                  = 2100  # final time
igm.config.tsave                 = 1     # saving frequence
igm.config.cfl                   = 0.25  # CFL condition must be lower than 1

igm.config.iceflow_model_lib_path= 'f15_cfsflow_GJ_22_a'  # direct to the iceflow emaultor
igm.config.type_climate          = 'aletsch'              # select type of climate forcing

igm.config.init_slidingco        = 0     # this parametrizes the ice flow (sliding), the higher this param, the faster the flow
igm.config.init_arrhenius        = 78    # this parametrizes the ice flow (shearing), the higher this param, the faster the flow

igm.config.type_mass_balance     = 'accmelt'              # select type of mass balance
igm.config.massbalance_file      = 'massbalance.nc'       # mass balance data
igm.config.weight_accumulation   = 1.00                   # this param weights accumulation
igm.config.weight_ablation       = 1.25                   # this param weights ablation

igm.config.usegpu                = True

igm.config.weight_Aletschfirn    = 1.0 # this param weights Aletschfirn accumulation area
igm.config.weight_Jungfraufirn   = 1.0 # this param weights Jungfraufirn accumulation area
igm.config.weight_Ewigschneefeld = 1.0 # this param weights Ewigschneefeld accumulation area

igm.initialize()

+++++++++++++++++++ START IGM ++++++++++++++++++++++++++++++++++++++++++
PARAMETERS ARE ...... 
                   working_dir : 
                  geology_file : geology.nc
                      resample : 1
                        tstart : 1880
                          tend : 2100
                restartingfile : 
                     verbosity : 0
                         tsave : 1
                   plot_result : False
                     plot_live : False
                        usegpu : True
                          stop : False
              init_strflowctrl : 78
                init_slidingco : 0
                init_arrhenius : 78
                      optimize : False
                   update_topg : False
                  vel3d_active : False
                            dz : 20
                        maxthk : 1000.0
               weight_ablation : 1.25
           weight_accumulation : 1.0
                 thr_temp_snow : 0.5
                 thr_temp_rain : 2.5
       

We are now ready to simulate the great Aletsch Glacier for 220 years from 1880. The next code could be substitute by the simple comand igm.run(), however, we use the full workflow as we included an assemement of the model output ice surface against observation in years 1880, 1926, 1957, 1980, 1999, 2009, 2017 and 2016.


In [4]:
with tf.device(igm.device_name):

    igm.load_ncdf_data(igm.config.geology_file)
    
    # load the surface toporgaphy available at given year
    igm.usurf.assign(vars(igm)['surf_'+str(int(igm.t))])
    igm.thk.assign(igm.usurf-igm.topg)
    
    igm.initialize_fields() 

    while igm.t < igm.config.tend:
        
        igm.tcomp["All"].append(time.time())
             
        # For thes year, check the std between modelled and observed surfaces
        if igm.t in [1926,1957,1980,1999,2009,2017]:
            diff = (igm.usurf-vars(igm)['surf_'+str(int(igm.t))]).numpy()
            diff = diff[igm.thk>1]
            mean  = np.mean(diff)
            std   = np.std(diff)
            vol   = np.sum(igm.thk) * (igm.dx ** 2) / 10 ** 9
            print(" Check modelled vs observed surface at time : %8.0f ; Mean discr. : %8.2f  ;  Std : %8.2f |  Ice volume : %8.2f " \
                  % (igm.t, mean, std, vol) )
        
        igm.update_climate()
        igm.update_smb() 
        igm.update_iceflow()
        igm.update_t_dt()
        igm.update_thk()
        igm.update_ncdf_ex()
        igm.update_ncdf_ts()
        igm.update_plot()
        igm.print_info()
        
        igm.tcomp["All"][-1] -= time.time()
        igm.tcomp["All"][-1] *= -1
        
    igm.print_all_comp_info()

# vizualize
igm.animate_result('ex.nc','thk',save=True)
igm.animate_result('ex.nc','velsurf_mag',save=True)

IGM 07:21:54 : Iterations =      0  |  Time =     1880  |  DT =   10.00  |  Ice Volume (km^3) =      20.67 
IGM 07:21:55 : Iterations =     14  |  Time =     1881  |  DT =    0.09  |  Ice Volume (km^3) =      20.62 
IGM 07:21:56 : Iterations =     26  |  Time =     1882  |  DT =    0.09  |  Ice Volume (km^3) =      20.51 
IGM 07:21:56 : Iterations =     38  |  Time =     1883  |  DT =    0.09  |  Ice Volume (km^3) =      20.57 
IGM 07:21:56 : Iterations =     49  |  Time =     1884  |  DT =    0.10  |  Ice Volume (km^3) =      20.55 
IGM 07:21:57 : Iterations =     60  |  Time =     1885  |  DT =    0.10  |  Ice Volume (km^3) =      20.44 
IGM 07:21:57 : Iterations =     70  |  Time =     1886  |  DT =    0.11  |  Ice Volume (km^3) =      20.35 
IGM 07:21:57 : Iterations =     80  |  Time =     1887  |  DT =    0.11  |  Ice Volume (km^3) =      20.28 
IGM 07:21:57 : Iterations =     90  |  Time =     1888  |  DT =    0.11  |  Ice Volume (km^3) =      20.16 
IGM 07:21:58 : Iterations = 

We have now modelled 220 years of evolution of the Great Aletsch Glacier at 100-meter resolution in about one minute. IGM monitors key variables (ice volume, time step) during computation, and provide STDs between modelled and observed ice surfaces in some years. Melt factors were tuned to keep this STD fairly small (below 35 m). The next code permits to vizualize the simulation.


In [5]:
igm.animate_result('ex.nc','thk',save=True)
#igm.animate_result('ex.nc','velsurf_mag',save=True)

CalledProcessError: ignored

In [ ]:
# always good to clean the space
!rm ts.nc ex.nc igm-run-parameters.txt

Let's now change the mass balance moddel, and use a well-trained neural network trained from climate and mass balance data (from Aletsch Glacier). This can be seen as an alternative to the accumulation / melt model. Therefore, you must set igm.config.type_mass_balance to 'nn', provide a valid path for the mass balance emulator igm.config.smb_model_lib_path, and also force monthly climate inputs (instead of daily) setting igm.config.clim_time_resolution to 12.

In [ ]:
# get the SMB emulator
!svn export https://github.com/jouvetg/igm/trunk/model-lib/smb1_meteoswissglamos_GJ_21_a

# option 2: emulated smb model by a CNN -- uncoment these lines
igm.config.smb_model_lib_path    = 'smb1_meteoswissglamos_GJ_21_a'
igm.config.type_mass_balance     = 'nn'
igm.config.clim_time_resolution  = 12
igm.config.tstart                = 2017
igm.config.tend                  = 2100

Then simply launch IGM time evolution:

In [ ]:
igm.run()

The simulation shows slightly higher volumes (compared to before), but does not lead to substantially different results.

In [7]:
# clean all
! rm -r f15_cfsflow_GJ_22_a smb1_meteoswissglamos_GJ_21_a __pycache__ ex-*.mp4 
! rm *.nc  *.py *.dat *.txt

rm: cannot remove 'f15_cfsflow_GJ_22_a': No such file or directory
rm: cannot remove 'smb1_meteoswissglamos_GJ_21_a': No such file or directory
rm: cannot remove '__pycache__': No such file or directory
rm: cannot remove '*.nc': No such file or directory
rm: cannot remove '*.py': No such file or directory
rm: cannot remove '*.dat': No such file or directory
rm: cannot remove '*.txt': No such file or directory
